# multiMentalRoBERTa Model Inference


### multiMentalRoBERTa Model Inference from HuggingFace

In [12]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("SajjadIslam/multiMentalRoBERTA-5-class")

print("Number of labels:", config.num_labels)
print("id2label:", config.id2label)
print("label2id:", config.label2id)


Number of labels: 5
id2label: {0: 'anxiety', 1: 'depression', 2: 'ptsd', 3: 'suicide', 4: 'none'}
label2id: {'anxiety': 0, 'depression': 1, 'none': 4, 'ptsd': 2, 'suicide': 3}


In [15]:
config = AutoConfig.from_pretrained("SajjadIslam/multiMentalRoBERTA-6-class")

print("Number of labels:", config.num_labels)
print("id2label:", config.id2label)
print("label2id:", config.label2id)


Number of labels: 6
id2label: {0: 'anxiety', 1: 'depression', 2: 'ptsd', 3: 'suicide', 4: 'stress', 5: 'none'}
label2id: {'anxiety': 0, 'depression': 1, 'none': 5, 'ptsd': 2, 'stress': 4, 'suicide': 3}


In [16]:
import os
from typing import Dict, Any

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 512

# Repo ids on the Hub
# If you later split into two repos, change these two strings
MODEL_NO_STRESS_REPO = "SajjadIslam/multiMentalRoBERTA-5-class"
MODEL_WITH_STRESS_REPO = "SajjadIslam/multiMentalRoBERTA-6-class"

print(f"Device: {DEVICE}")
print(f"Using Hub repos: {MODEL_NO_STRESS_REPO} | {MODEL_WITH_STRESS_REPO}")

# Load models and tokenizers from the Hub
tokenizer_no_stress = AutoTokenizer.from_pretrained(MODEL_NO_STRESS_REPO, use_fast=True)
model_no_stress = AutoModelForSequenceClassification.from_pretrained(MODEL_NO_STRESS_REPO).to(DEVICE)
model_no_stress.eval()

tokenizer_with_stress = AutoTokenizer.from_pretrained(MODEL_WITH_STRESS_REPO, use_fast=True)
model_with_stress = AutoModelForSequenceClassification.from_pretrained(MODEL_WITH_STRESS_REPO).to(DEVICE)
model_with_stress.eval()

# Build id2label from config, fall back to numeric labels if missing
def get_id2label(m) -> Dict[int, str]:
    cfg = m.config
    if getattr(cfg, "id2label", None) and len(cfg.id2label) == cfg.num_labels:
        # keys may be strings in some configs
        return {int(k): v for k, v in cfg.id2label.items()}
    return {i: f"LABEL_{i}" for i in range(cfg.num_labels)}

id2label_no_stress = get_id2label(model_no_stress)
id2label_with_stress = get_id2label(model_with_stress)

print("Models loaded successfully from the Hub!")

@torch.no_grad()
def classify(text: str, model_name: str = "no_stress") -> Dict[str, Any]:
    """
    Classify text using a Hub model.

    Args:
        text: input text
        model_name: "no_stress" or "with_stress"

    Returns:
        dict with predicted_class, confidence, probabilities
    """
    if model_name == "no_stress":
        model = model_no_stress
        tokenizer = tokenizer_no_stress
        id2label = id2label_no_stress
    else:
        model = model_with_stress
        tokenizer = tokenizer_with_stress
        id2label = id2label_with_stress

    encoded = tokenizer(
        str(text),
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH,
        return_tensors="pt",
    ).to(DEVICE)

    outputs = model(**encoded)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]

    predicted_id = int(torch.argmax(logits, dim=-1).item())
    predicted_class = id2label[predicted_id]

    return {
        "predicted_class": predicted_class,
        "confidence": float(probs[predicted_id]),
        "probabilities": {id2label[i]: float(probs[i]) for i in range(len(probs))},
    }

# Example
if __name__ == "__main__":
    text = "I have been feeling really down lately and do not have the energy to do anything."
    result = classify(text, model_name="no_stress")
    print(f"Predicted: {result['predicted_class']}")
    print(f"Confidence: {result['confidence']:.4f}")


Device: cuda
Using Hub repos: SajjadIslam/multiMentalRoBERTA-5-class | SajjadIslam/multiMentalRoBERTA-6-class


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


Models loaded successfully from the Hub!
Predicted: depression
Confidence: 0.9625


### multiMentalRoBERTa Model Inference from Local Directory

Classify text using:
- `all_roberta_large_v1_multiclass` (5 classes)
- `all_roberta_large_v1_with_stress` (6 classes)

In [1]:
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForSequenceClassification

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 512


c:\Users\nipua\AppData\Local\anaconda3\envs\py312_gpu_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Find and load models
def find_data_warehouse():
    for p in [Path.cwd()] + list(Path.cwd().parents):
        dw = p / "Data_Warehouse"
        if dw.exists():
            return dw
    raise FileNotFoundError("Could not locate Data_Warehouse")

DATA_WAREHOUSE = find_data_warehouse()
MODEL_NO_STRESS = DATA_WAREHOUSE / "mental_health_splits_no_stress" / "all_roberta_large_v1_multiclass" / "best"
MODEL_WITH_STRESS = DATA_WAREHOUSE / "mental_health_splits_with_stress" / "all_roberta_large_v1_with_stress" / "best"

if not MODEL_NO_STRESS.exists():
    MODEL_NO_STRESS = MODEL_NO_STRESS.parent
if not MODEL_WITH_STRESS.exists():
    MODEL_WITH_STRESS = MODEL_WITH_STRESS.parent

print(f"Device: {DEVICE}")
print(f"Loading models from: {DATA_WAREHOUSE}")


Device: cuda
Loading models from: d:\Sajjad-Workspace\PSS_XAI\Data_Process\Data_Warehouse


In [3]:
# Load models
tokenizer_no_stress = AutoTokenizer.from_pretrained(MODEL_NO_STRESS, use_fast=True)
model_no_stress = AutoModelForSequenceClassification.from_pretrained(MODEL_NO_STRESS).to(DEVICE)
model_no_stress.eval()

tokenizer_with_stress = AutoTokenizer.from_pretrained(MODEL_WITH_STRESS, use_fast=True)
model_with_stress = AutoModelForSequenceClassification.from_pretrained(MODEL_WITH_STRESS).to(DEVICE)
model_with_stress.eval()

id2label_no_stress = model_no_stress.config.id2label
id2label_with_stress = model_with_stress.config.id2label

print("Models loaded successfully!")


Models loaded successfully!


In [4]:
@torch.no_grad()
def classify(text, model_name="no_stress"):
    """
    Classify text.
    
    Args:
        text: Input text to classify
        model_name: "no_stress" (5 classes) or "with_stress" (6 classes)
    
    Returns:
        Dictionary with 'predicted_class', 'confidence', and 'probabilities'
    """
    if model_name == "no_stress":
        model = model_no_stress
        tokenizer = tokenizer_no_stress
        id2label = id2label_no_stress
    else:
        model = model_with_stress
        tokenizer = tokenizer_with_stress
        id2label = id2label_with_stress
    
    encoded = tokenizer(
        str(text),
        truncation=True,
        padding='max_length',
        max_length=MAX_LENGTH,
        return_tensors='pt'
    ).to(DEVICE)
    
    outputs = model(**encoded)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1).cpu().numpy()[0]
    
    predicted_id = int(torch.argmax(logits, dim=-1).item())
    predicted_class = id2label[predicted_id]
    
    return {
        'predicted_class': predicted_class,
        'confidence': float(probs[predicted_id]),
        'probabilities': {id2label[i]: float(probs[i]) for i in range(len(probs))}
    }


In [ ]:
# Classify text
# result = classify("Your text here", model_name="no_stress")  # or "with_stress"


In [6]:
# Example usage
text = "I've been feeling really down lately and don't have the energy to do anything."
result = classify(text, model_name="no_stress")
print(f"Predicted: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.4f}")


Predicted: depression
Confidence: 0.9814


In [7]:
# Example with stress model
text = "I'm so stressed about my exams and I can't sleep at night."
result = classify(text, model_name="with_stress")
print(f"Predicted: {result['predicted_class']}")
print(f"Confidence: {result['confidence']:.4f}")


Predicted: stress
Confidence: 0.9990
